# Problem Set 10
## Daryl Larsen

### 2 (MC simulations for Lasso)

In [1]:
using Random, Distributions, PrettyTables, SCS, Plots, LinearAlgebra, Convex
using Base.Threads: @threads, @spawn
Random.seed!(220407)

MersenneTwister(220407)

In [2]:
const k = 50;
const α = 1;
const n = 300;

### a) Generate the data

In [3]:
function dgp(ρ, σ, β1, β2)
    U = rand(Normal(0, σ^2), n)
    cov = fill(ρ, (k,k))
    cov[diagind(cov)] .= 1
    X = rand(MvNormal(zeros(50), cov),n)'
    
    Y = α .+ β1*X[:,1] .+ β2*X[:,2] .+ U
    
    return (Y = Y, X = X)
end

dgp (generic function with 1 method)

### b) Compute Lasso estimates

In [4]:
MOI=Convex.MOI
opt = MOI.OptimizerWithAttributes(SCS.Optimizer, "eps_abs"=>1.0e-08,"eps_rel"=>1.0e-08, MOI.Silent() => true)

function Lasso(Y, X, λ)
    N,K = size(X) 
    X=[X ones(N,1)]
    
    b = Variable(K+1) 
    
    Q=X'X/N;
    c=X'Y/N;
    
    L1=quadform(b, Q)/2  
    L2 = dot(c, b) 
    L3 = norm(b[1:K], 1)

    Sol=minimize(L1-L2+λ*L3)
    
    solve!(Sol, opt, verbose=false)
    
    estr=round.(vec(evaluate(b)),digits=5)

    return estr
end

Lasso (generic function with 1 method)

### c) Run 10k Monte Carlo simulations

In [5]:
function sims(ρ, σ, β1, β2)

    Y,X = dgp(ρ,σ,β1,β2)

    λ = 1.1 * σ * sqrt((2 * log(n * k)) / n)

    b_i = Lasso(Y, X, λ)
    
    count_1 = 0
    count_1 += b_i[1] != 0 ? 1.0 : 0.0

    count_2 = 0
    count_2 += b_i[2] != 0 ? 1.0 : 0.0

    count_3 = 0
    count_3 +=  sum((b_i[3:50] .!= 0) ) > 0 ? 1.0 : 0.0

    return (count_1 = count_1, count_2 = count_2, count_3 = count_3)
end

sims (generic function with 1 method)

In [6]:
sto_1c = 0
sto_2c = 0
sto_3c = 0

R = 10000

for i = 1:R

    count_1, count_2, count_3 = sims(0.2,2.0,2.0,2.0)
    sto_1c += count_1
    sto_2c += count_2
    sto_3c += count_3

end

sto_1c = sto_1c / R
sto_2c = sto_2c / R
sto_3c = sto_3c / R

0.7783

### d)

In [7]:
sto_1d = 0
sto_2d = 0
sto_3d = 0

for i = 1:R

    count_1, count_2, count_3 = sims(0.2,0.1,0.1,0.1)
    sto_1d += count_1
    sto_2d += count_2
    sto_3d += count_3

end

sto_1d = sto_1d / R
sto_2d = sto_2d / R
sto_3d = sto_3d / R

0.0

### e)

In [8]:
sto_1e = 0
sto_2e = 0
sto_3e = 0

for i = 1:R

    count_1, count_2, count_3 = sims(0.2,2.0,2.0,0.2)
    sto_1e += count_1
    sto_2e += count_2
    sto_3e += count_3

end

sto_1e = sto_1e / R
sto_2e = sto_2e / R
sto_3e = sto_3e / R

0.6627

### f)

In [9]:
sto_1f = 0
sto_2f = 0
sto_3f = 0

for i = 1:R

    count_1, count_2, count_3 = sims(0.9,2.0,2.0,2.0)
    sto_1f += count_1
    sto_2f += count_2
    sto_3f += count_3

end

sto_1f = sto_1f / R
sto_2f = sto_2f / R
sto_3f = sto_3f / R

0.9896

### g)

In [10]:
table_header = ["Specification","Prob 1", "Prob 2", "Prob 3"]
content = ["c)" sto_1c sto_2c sto_3c ; "d)" sto_1d sto_2d sto_3d ; "e)" sto_1e sto_2e sto_3e ; "f)" sto_1f sto_2f sto_3f]
pretty_table(content, table_header)

┌───────────────┬────────┬────────┬────────┐
│ Specification │ Prob 1 │ Prob 2 │ Prob 3 │
├───────────────┼────────┼────────┼────────┤
│            c) │    1.0 │    1.0 │ 0.7783 │
│            d) │    1.0 │    1.0 │    0.0 │
│            e) │    1.0 │ 0.1093 │ 0.6627 │
│            f) │ 0.9292 │ 0.9279 │ 0.9896 │
└───────────────┴────────┴────────┴────────┘
